## Import and login

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

catalog = SentinelHubCatalog(config=config)

Enter your SentinelHub client id········
Enter your SentinelHub client secret········


## Evaluation scripts

In [2]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B2","B3","B4"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B4,sample.B3,sample.B2];
    }
"""

evalscript_B12 = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B12"]
            }],
            output: {
                bands: 1
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B12];
    }
"""

evalscript_B2 = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B2"]
            }],
            output: {
                bands: 1
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B2];
    }
"""

Class **war_zone** ...

Problem with ***plot_grid***, something fishy about the grid lines, but it's very secondary at this point

Function ***generate***

In [ ]:
import datetime

class war_zone:
    
    def __init__(self,east,south,west,north,crs=CRS.WGS84,resolution=10):
        self.bounding_box=[east,south,west,north]
        self.crs=crs
        self.resolution=resolution
        self.aoi_size = bbox_to_dimensions(BBox(bbox=self.bounding_box, crs=self.crs), resolution=self.resolution)
        self.easts = self.dim_to_2500(self.aoi_size[0])
        self.souths = self.dim_to_2500(self.aoi_size[1],NS=True)
        self.data_true_color={}
        self.data_B12={}
        self.data_B2={}
        #print(self.bounding_box)
        ##print(self.easts)
        #print(self.souths)
        #for e in self.easts:
        #    print(self.aoi_size[0]*(e-self.bounding_box[0])/(self.bounding_box[2]-self.bounding_box[0]))
        #print("s")
        #for s in self.souths:
        #    print(self.aoi_size[1]*(self.bounding_box[3]-s)/(self.bounding_box[3]-self.bounding_box[1]))
  
        
    def dim_to_2500(self,dim,NS=False):
        ind=int(NS)
        scl=dim/2500
        if dim>2500:
            subdivide=int(scl)+2
        else:
            subdivide=1
        dim=(self.bounding_box[2+ind]-self.bounding_box[ind])/scl
        shift=(subdivide%2)/2
        return [(self.bounding_box[ind]+self.bounding_box[2+ind])/2+(xi-shift)*dim for xi in range(-int(subdivide/2),int(subdivide/2)+1+(subdivide%2))]
            
    def plot_grid(self,ax,color="black",map_to_image=True,linewidth=10):
        for e in self.easts[1:-1]:
            if map_to_image:
                ax.axvline(x=self.aoi_size[0]*(e-self.bounding_box[0])/(self.bounding_box[2]-self.bounding_box[0]),color=color,linewidth=linewidth)
            else:
                ax.axvline(x=e,color=color,linewidth=linewidth)
        for s in self.souths[1:-1]:
            if map_to_image:
                ax.axhline(y=self.aoi_size[1]*(s-self.bounding_box[1])/(self.bounding_box[3]-self.bounding_box[1]),color=color,linewidth=linewidth)
            else:
                ax.axhline(y=s,color=color,linewidth=linewidth)
            
    def check_availability_for_date(self,date):
        aoi_bbox = BBox(bbox=self.bounding_box, crs=CRS.WGS84)
        time_interval = date, str(datetime.date(int(date[0:4]),int(date[5:7]),int(date[8:10]))+datetime.timedelta(days=1))
        search_iterator = catalog.search(DataCollection.SENTINEL2_L2A,bbox=aoi_bbox,time=time_interval,fields={"include": ["id", "properties.datetime"], "exclude": []},)
        if len(list(search_iterator))>0:
            return time_interval, True
        else:
            return time_interval, False
        
    def download(self,date,typ="B12"):
        if typ=="true_color":
            script=evalscript_true_color
        elif typ=="B12":
            script=evalscript_B12
        elif typ=="B2":
            script=evalscript_B2
        time_interval, availability = self.check_availability_for_date(date)
        if not availability:
            print("No data for date {}".format(date))
        else:
            im = []
            for j in range(len(self.souths)-1):
                row=[]
                for i in range(len(self.easts)-1):
                    aoi_bbox = BBox(bbox=[self.easts[i],self.souths[j],self.easts[i+1],self.souths[j+1]], crs=self.crs)
                    request = SentinelHubRequest(evalscript=script,input_data=[SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A.define_from(name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"), time_interval=(time_interval),other_args={"dataFilter": {"mosaickingOrder": "leastCC"}})],responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],bbox=aoi_bbox,size=(2500,2500),config=config)
                    image=request.get_data()[0]
                    row.append(image)
                im=[np.concatenate(list(r for r in row),axis=1)]+im
            final_im=np.concatenate(list(c for c in im))
            final_im=final_im[int(self.aoi_size[0]*(self.bounding_box[0]-self.easts[0])/(self.easts[-1]-self.easts[0])):-int(self.aoi_size[0]*(self.easts[-1]-self.bounding_box[2])/(self.easts[-1]-self.easts[0])),int(self.aoi_size[1]*(self.bounding_box[1]-self.souths[0])/(self.souths[-1]-self.souths[0])):-int(self.aoi_size[1]*(self.souths[-1]-self.bounding_box[3])/(self.souths[-1]-self.souths[0]))]
            if typ=="true_color":
                self.data_true_color[date]=final_im
            elif typ=="B12":
                self.data_B12[date]=final_im
            elif typ=="B2":
                self.data_B2[date]=final_im
                
    def plot(self,date,factor=100,typ="B12",with_grid=False,grid_color="black",save_to=None,threshhold=None):
        figsize = np.array(self.aoi_size)/factor
        if figsize[0]>=2**16:
            figsize/=(figsize[0]/2**17)  
        if figsize[1]>=2**16:
            figsize/=(figsize[1]/2**17)
        fig, ax = plt.subplots(figsize=(figsize[0],figsize[1]))
        if typ=="true_color":
            ax.imshow(self.data_true_color[date])
        elif typ=="B12":
            if threshhold is None:
                ax.imshow(self.data_B12[date],cmap="gray")
            else:
                ax.imshow(self.data_B12[date]>threshhold,cmap="gray")
        elif typ=="B2":
            ax.imshow(self.data_B2[date],cmap="gray")
        if with_grid:
            self.plot_grid(ax,color=grid_color)
        if not save_to is None:
            fig.savefig(save_to)
            plt.close()
            
    def generate_matrix_of_weights_based_on_sentinel(self):
        return